In [1]:
%reload_ext autotime
import geopandas as gpd
import pandas as pd
from glob import glob
from sklearn.linear_model import LinearRegression
from tqdm.auto import tqdm
from tqdm.contrib.concurrent import process_map
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, root_mean_squared_error
from coastsat import SDS_transects
pd.options.plotting.backend = "plotly"

In [2]:
# Transects, origin is landward. Has beach_slope
transects = gpd.read_file("transects_extended.geojson")
transects.set_index("id", inplace=True)
transects

site_id  orientation  along_dist  along_dist_norm  beach_slope  \
id                                                                             
aus0001-0000  aus0001   104.347648    0.000000         0.000000        0.085   
aus0001-0001  aus0001    93.495734   98.408334         0.002935        0.050   
aus0001-0002  aus0001    82.069341  198.408334         0.005918        0.050   
aus0001-0003  aus0001    81.192757  298.402523         0.008900        0.055   
aus0001-0004  aus0001    81.065473  398.402523         0.011882        0.075   
...               ...          ...         ...              ...          ...   
ber0002-0009  ber0002          NaN         NaN              NaN          NaN   
ber0002-0010  ber0002          NaN         NaN              NaN          NaN   
ber0002-0011  ber0002          NaN         NaN              NaN          NaN   
ber0002-0012  ber0002          NaN         NaN              NaN          NaN   
ber0002-0013  ber0002          NaN         NaN              NaN          NaN   

                 cil     ciu     trend  n_points  n_points_nonan  r2_score  \
id                                                                           
aus0001-0000  0.0545  0.2000 -1.441081     767.0           428.0  0.168420   
aus0001-0001  0.0387  0.0640 -1.037105     767.0           569.0  0.097874   
aus0001-0002  0.0428  0.0647 -0.680019     767.0           588.0  0.053927   
aus0001-0003  0.0480  0.0659 -0.405198     767.0           598.0  0.023412   
aus0001-0004  0.0614  0.0922 -0.090025     767.0           608.0  0.001277   
...              ...     ...       ...       ...             ...       ...   
ber0002-0009     NaN     NaN       NaN       NaN             NaN       NaN   
ber0002-0010     NaN     NaN       NaN       NaN             NaN       NaN   
ber0002-0011     NaN     NaN       NaN       NaN             NaN       NaN   
ber0002-0012     NaN     NaN       NaN       NaN             NaN       NaN   
ber0002-0013     NaN     NaN       NaN       NaN             NaN       NaN   

                    mae          mse       rmse   intercept ERODIBILITY  \
id                                                                        
aus0001-0000  28.102591  1263.560863  35.546601  179.085729        None   
aus0001-0001  25.419324  1033.770813  32.152306  212.247788        None   
aus0001-0002  22.632907   838.007507  28.948359  205.106151        None   
aus0001-0003  20.749758   698.653187  26.432048  191.745881        None   
aus0001-0004  19.889328   655.810616  25.608800  175.092121        None   
...                 ...          ...        ...         ...         ...   
ber0002-0009        NaN          NaN        NaN         NaN        None   
ber0002-0010        NaN          NaN        NaN         NaN        None   
ber0002-0011        NaN          NaN        NaN         NaN        None   
ber0002-0012        NaN          NaN        NaN         NaN        None   
ber0002-0013        NaN          NaN        NaN         NaN        None   

                                                       geometry  
id                                                               
aus0001-0000  LINESTRING (153.26555 -24.7007, 153.26938 -24....  
aus0001-0001  LINESTRING (153.26525 -24.7019, 153.2692 -24.7...  
aus0001-0002  LINESTRING (153.26539 -24.70316, 153.26931 -24...  
aus0001-0003  LINESTRING (153.26555 -24.70408, 153.26945 -24...  
aus0001-0004  LINESTRING (153.2657 -24.70497, 153.26961 -24....  
...                                                         ...  
ber0002-0009  LINESTRING (-64.82204 32.25336, -64.82017 32.2...  
ber0002-0010  LINESTRING (-64.82143 32.25361, -64.82029 32.2...  
ber0002-0011  LINESTRING (-64.82118 32.25369, -64.82004 32.2...  
ber0002-0012  LINESTRING (-64.82049 32.25382, -64.82022 32.2...  
ber0002-0013  LINESTRING (-64.82022 32.25384, -64.81995 32.2...  

[147216 rows x 17 columns]

In [3]:
vos_files = pd.Series(
    sorted(glob("csv_run7/*/time_series_tidally_corrected.csv"))
)
vos_files = vos_files[~vos_files.str.contains("nzd")]
vos_files

0       csv_run7/aus0001/time_series_tidally_corrected...
1       csv_run7/aus0002/time_series_tidally_corrected...
2       csv_run7/aus0003/time_series_tidally_corrected...
3       csv_run7/aus0004/time_series_tidally_corrected...
4       csv_run7/aus0005/time_series_tidally_corrected...
                              ...                        
3066    csv_run7/usa_HI_0053/time_series_tidally_corre...
3067    csv_run7/usa_HI_0054/time_series_tidally_corre...
3068    csv_run7/usa_HI_0055/time_series_tidally_corre...
3069    csv_run7/usa_HI_0056/time_series_tidally_corre...
3070    csv_run7/usa_HI_0057/time_series_tidally_corre...
Length: 2511, dtype: object

In [28]:
my_files = pd.Series(
    sorted(glob("data/*/transect_time_series_tidally_corrected.csv"))
)
my_files

0      data/ber0001/transect_time_series_tidally_corr...
1      data/ber0002/transect_time_series_tidally_corr...
2      data/nzd0001/transect_time_series_tidally_corr...
3      data/nzd0002/transect_time_series_tidally_corr...
4      data/nzd0003/transect_time_series_tidally_corr...
                             ...                        
559    data/nzd0559/transect_time_series_tidally_corr...
560    data/nzd0560/transect_time_series_tidally_corr...
561    data/nzd0561/transect_time_series_tidally_corr...
562    data/nzd0562/transect_time_series_tidally_corr...
563    data/nzd0563/transect_time_series_tidally_corr...
Length: 564, dtype: object

In [5]:
sar_files = pd.Series(sorted(glob("data/sar*/transect_time_series.csv")))
sar_files

0       data/sar0001/transect_time_series.csv
1       data/sar0002/transect_time_series.csv
2       data/sar0003/transect_time_series.csv
3       data/sar0004/transect_time_series.csv
4       data/sar0005/transect_time_series.csv
                        ...                  
2536    data/sar2537/transect_time_series.csv
2537    data/sar2538/transect_time_series.csv
2538    data/sar2539/transect_time_series.csv
2539    data/sar2540/transect_time_series.csv
2540    data/sar2541/transect_time_series.csv
Length: 2541, dtype: object

In [6]:
files = pd.concat([vos_files, my_files, sar_files])
files

0       csv_run7/aus0001/time_series_tidally_corrected...
1       csv_run7/aus0002/time_series_tidally_corrected...
2       csv_run7/aus0003/time_series_tidally_corrected...
3       csv_run7/aus0004/time_series_tidally_corrected...
4       csv_run7/aus0005/time_series_tidally_corrected...
                              ...                        
2536                data/sar2537/transect_time_series.csv
2537                data/sar2538/transect_time_series.csv
2538                data/sar2539/transect_time_series.csv
2539                data/sar2540/transect_time_series.csv
2540                data/sar2541/transect_time_series.csv
Length: 5615, dtype: object

In [7]:
f = files[files.str.contains("sar1026")].iloc[0]
# despiked_filename = f.replace(".csv", "_tidally_corrected.csv")
df = pd.read_csv(f)
df.dates = pd.to_datetime(df.dates)
df.set_index("dates", inplace=True)
display(df.columns)
import matplotlib.pyplot as plt

transect_id = "sar1026-0007"


def custom_mean(window):
    return window[window.between(window.quantile(0.25), window.quantile(0.75))].mean()


pd.DataFrame(
    {
        "raw": df[transect_id],
        "rolling 90d mean": df[transect_id].rolling("90d", min_periods=1).mean(),
        "rolling 180d mean": df[transect_id].rolling("180d", min_periods=1).mean(),
        "rolling 90d custom mean": df[transect_id]
        .rolling("90d", min_periods=1)
        .apply(custom_mean),
        "rolling 180d custom mean": df[transect_id]
        .rolling("180d", min_periods=1)
        .apply(custom_mean),
        # "rolling 365d": df[transect_id].rolling("365d", min_periods=1).mean(),
    },
    index=df.index,
).plot()

Index(['satname', 'sar1026-0000', 'sar1026-0001', 'sar1026-0002',
       'sar1026-0003', 'sar1026-0004', 'sar1026-0005', 'sar1026-0006',
       'sar1026-0007'],
      dtype='object')

In [8]:
df = pd.read_csv("data/sar0939/transect_time_series.csv")
df.dates = pd.to_datetime(df.dates)
df.set_index("dates", inplace=True)
(df["sar0939-0000"] - 93).plot()

In [29]:
def despike(chainage, threshold=40):
    chainage = chainage.dropna()
    chainage, dates = SDS_transects.identify_outliers(
        chainage.tolist(), chainage.index.tolist(), threshold
    )
    return pd.Series(chainage, index=dates)


def get_trends(f):
    df = pd.read_csv(f)
    try:
        df.dates = pd.to_datetime(df.dates)
    except:
        print(f)
    if "sar" in f:
        smoothed_filename = f.replace(".csv", "_smoothed.csv")
        try:
            df = pd.read_csv(smoothed_filename)
            df.dates = pd.to_datetime(df.dates)
        except:
            df.dates = pd.to_datetime(df.dates)
            df.set_index("dates", inplace=True)
            satname = df.satname
            df = df.drop(columns="satname").apply(despike, axis=0)
            df["satname"] = satname
            df.reset_index(names="dates").to_csv(
                f.replace(".csv", "_despiked.csv"), index=False
            )
            for transect_id in df.drop(columns="satname").columns:
                df[transect_id] = df[transect_id].rolling("180d", min_periods=1).mean()
            df.reset_index(names="dates", inplace=True)
            df.to_csv(f.replace(".csv", "_smoothed.csv"), index=False)
    df.index = (df.dates - df.dates.min()).dt.days / 365.25
    df.drop(columns=["dates", "satname", "Unnamed: 0"], inplace=True, errors="ignore")
    trends = []
    for transect_id in df.columns:
        sub_df = df[transect_id].dropna()
        if not len(sub_df):
            continue
        x = sub_df.index.to_numpy().reshape(-1, 1)
        y = sub_df
        linear_model = LinearRegression().fit(x, y)
        pred = linear_model.predict(x)
        trends.append(
            {
                "transect_id": transect_id,
                "trend": linear_model.coef_[0],
                "intercept": linear_model.intercept_,
                "n_points": len(df[transect_id]),
                "n_points_nonan": len(sub_df),
                "r2_score": r2_score(y, pred),
                "mae": mean_absolute_error(y, pred),
                "mse": mean_squared_error(y, pred),
                "rmse": root_mean_squared_error(y, pred),
            }
        )
    return pd.DataFrame(trends)


# trends = get_trends(sar_files.iloc[-1]).set_index("transect_id")
trends = pd.concat(process_map(get_trends, my_files)).set_index("transect_id")
len(trends)

  0%|          | 0/564 [00:00<?, ?it/s]

32373

In [25]:
trends[trends.n_points_nonan > 10].sort_values("r2_score")

trend   intercept  n_points  n_points_nonan      r2_score  \
transect_id                                                                   
nzd0454-0235  -0.000014  339.463241       356             234  1.108120e-10   
nzd0247-0024   0.000021  341.746233       384             348  2.646839e-10   
nzd0096-0005   0.000075  367.077323       292             197  6.685417e-10   
nzd0430-0082   0.000065  325.161591       622             488  1.516113e-09   
nzd0332-0065  -0.000071  358.663499       484             374  3.403982e-09   
...                 ...         ...       ...             ...           ...   
nzd0121-0001  15.042364  275.340274       171             107  9.424102e-01   
nzd0121-0003  14.505805  243.808848       171             115  9.433136e-01   
nzd0121-0002  15.309330  250.088130       171             111  9.438529e-01   
nzd0121-0005  14.302793  208.178540       171             114  9.487553e-01   
nzd0121-0006  14.228322  190.866580       171             114  9.491833e-01   

                    mae         mse       rmse  
transect_id                                     
nzd0454-0235   7.395218   92.562606   9.620946  
nzd0247-0024   7.312583   87.103013   9.332899  
nzd0096-0005  17.207332  457.713592  21.394242  
nzd0430-0082   9.667038  163.731264  12.795752  
nzd0332-0065   7.379230   86.335054   9.291666  
...                 ...         ...        ...  
nzd0121-0001  21.977610  794.467914  28.186307  
nzd0121-0003  19.881359  680.947223  26.094965  
nzd0121-0002  22.016239  777.948935  27.891736  
nzd0121-0005  20.292915  621.764706  24.935210  
nzd0121-0006  19.622221  593.817392  24.368369  

[32358 rows x 8 columns]

In [31]:
trends[trends.index.str.startswith("ber")]

trend   intercept  n_points  n_points_nonan  r2_score  \
transect_id                                                              
ber0001-0000 -0.448258  172.507072       237             198  0.242076   
ber0001-0001 -0.371049  164.008430       237             209  0.159783   
ber0001-0002 -0.231742  162.145915       237             211  0.066675   
ber0001-0003 -0.207635  162.591131       237             212  0.062541   
ber0001-0004 -0.213923  162.793609       237             212  0.053796   
ber0001-0005 -0.166451  160.122353       237             213  0.034842   
ber0001-0006 -0.070853  156.988174       237             213  0.005624   
ber0001-0007 -0.140678  160.533241       237             213  0.031743   
ber0001-0008 -0.403365  166.723593       237             211  0.303091   
ber0001-0009 -0.338969  165.941405       237             213  0.205309   
ber0002-0000  0.224665  145.439125       223             190  0.020157   
ber0002-0001 -0.097505  125.299373       223             188  0.006084   
ber0002-0002 -0.045765  125.112308       223             187  0.001488   
ber0002-0003 -0.129220  126.783486       223             189  0.012803   
ber0002-0004 -0.079790  124.573256       223             189  0.005916   
ber0002-0005 -0.034732  124.371637       223             190  0.001127   
ber0002-0006  0.004886  124.961954       223             190  0.000020   
ber0002-0007  0.025093  124.528243       223             191  0.000483   
ber0002-0008  0.011266  126.144871       223             193  0.000089   
ber0002-0009  0.088578  128.134846       223             192  0.006440   
ber0002-0010  0.028300  128.837353       223             190  0.000497   
ber0002-0011  0.057356  129.482491       223             187  0.002423   
ber0002-0012 -0.010553  131.010602       223             186  0.000081   
ber0002-0013  0.157248  136.403294       223             177  0.014649   

                   mae         mse       rmse  
transect_id                                    
ber0001-0000  4.656699   40.850800   6.391463  
ber0001-0001  5.105021   44.732802   6.688258  
ber0001-0002  5.086145   46.306011   6.804852  
ber0001-0003  4.598057   39.970386   6.322214  
ber0001-0004  5.345254   50.109808   7.078828  
ber0001-0005  5.368925   47.897882   6.920830  
ber0001-0006  5.767827   55.391345   7.442536  
ber0001-0007  4.806025   37.665152   6.137194  
ber0001-0008  3.511536   23.238632   4.820646  
ber0001-0009  3.766229   27.723053   5.265269  
ber0002-0000  9.291492  138.937996  11.787196  
ber0002-0001  7.326147   89.735897   9.472903  
ber0002-0002  6.751371   81.858417   9.047564  
ber0002-0003  6.567358   73.383543   8.566420  
ber0002-0004  5.976532   60.872208   7.802064  
ber0002-0005  6.066137   60.773925   7.795763  
ber0002-0006  6.374614   67.021602   8.186672  
ber0002-0007  6.816909   74.648560   8.639940  
ber0002-0008  7.237888   81.192587   9.010693  
ber0002-0009  6.645872   70.165869   8.376507  
ber0002-0010  7.682399   92.744157   9.630377  
ber0002-0011  7.512558   79.234887   8.901398  
ber0002-0012  7.210548   79.289497   8.904465  
ber0002-0013  7.547493   95.120607   9.752979

In [32]:
trends.describe()

trend     intercept      n_points  n_points_nonan      r2_score  \
count  32373.000000  32373.000000  32373.000000    32373.000000  3.237300e+04   
mean       0.122125    351.320976    437.676984      323.536713  1.477509e-01   
std        1.885357     55.016975    153.353424      118.179961  1.900329e-01   
min      -40.158498   -174.937352    102.000000       10.000000  1.108120e-10   
25%       -0.392536    328.001798    335.000000      244.000000  1.417269e-02   
50%        0.077002    349.026385    420.000000      317.000000  6.561779e-02   
75%        0.576606    372.136237    559.000000      412.000000  2.046366e-01   
max       29.201280   1138.442745    811.000000      711.000000  9.491833e-01   

                mae           mse          rmse  
count  32373.000000  32373.000000  32373.000000  
mean      12.565337    363.209583     16.027149  
std        8.362621    805.358607     10.312292  
min        3.511536     23.238632      4.820646  
25%        8.251138    111.947181     10.580510  
50%       10.316860    172.828326     13.146419  
75%       13.746437    305.039784     17.465388  
max      118.288420  20472.735372    143.082967

In [33]:
(transects.trend - trends.trend).describe()

count    32381.000000
mean         0.000079
std          0.004995
min         -0.011902
25%          0.000000
50%          0.000000
75%          0.000000
max          0.414383
Name: trend, dtype: float64

In [34]:
transects.update(trends.drop_duplicates())

In [35]:
transects

site_id  orientation  along_dist  along_dist_norm  beach_slope  \
id                                                                             
aus0001-0000  aus0001   104.347648    0.000000         0.000000        0.085   
aus0001-0001  aus0001    93.495734   98.408334         0.002935        0.050   
aus0001-0002  aus0001    82.069341  198.408334         0.005918        0.050   
aus0001-0003  aus0001    81.192757  298.402523         0.008900        0.055   
aus0001-0004  aus0001    81.065473  398.402523         0.011882        0.075   
...               ...          ...         ...              ...          ...   
ber0002-0009  ber0002          NaN         NaN              NaN          NaN   
ber0002-0010  ber0002          NaN         NaN              NaN          NaN   
ber0002-0011  ber0002          NaN         NaN              NaN          NaN   
ber0002-0012  ber0002          NaN         NaN              NaN          NaN   
ber0002-0013  ber0002          NaN         NaN              NaN          NaN   

                 cil     ciu     trend  n_points  n_points_nonan  r2_score  \
id                                                                           
aus0001-0000  0.0545  0.2000 -1.441081     767.0           428.0  0.168420   
aus0001-0001  0.0387  0.0640 -1.037105     767.0           569.0  0.097874   
aus0001-0002  0.0428  0.0647 -0.680019     767.0           588.0  0.053927   
aus0001-0003  0.0480  0.0659 -0.405198     767.0           598.0  0.023412   
aus0001-0004  0.0614  0.0922 -0.090025     767.0           608.0  0.001277   
...              ...     ...       ...       ...             ...       ...   
ber0002-0009     NaN     NaN  0.088578     223.0           192.0  0.006440   
ber0002-0010     NaN     NaN  0.028300     223.0           190.0  0.000497   
ber0002-0011     NaN     NaN  0.057356     223.0           187.0  0.002423   
ber0002-0012     NaN     NaN -0.010553     223.0           186.0  0.000081   
ber0002-0013     NaN     NaN  0.157248     223.0           177.0  0.014649   

                    mae          mse       rmse   intercept ERODIBILITY  \
id                                                                        
aus0001-0000  28.102591  1263.560863  35.546601  179.085729        None   
aus0001-0001  25.419324  1033.770813  32.152306  212.247788        None   
aus0001-0002  22.632907   838.007507  28.948359  205.106151        None   
aus0001-0003  20.749758   698.653187  26.432048  191.745881        None   
aus0001-0004  19.889328   655.810616  25.608800  175.092121        None   
...                 ...          ...        ...         ...         ...   
ber0002-0009   6.645872    70.165869   8.376507  128.134846        None   
ber0002-0010   7.682399    92.744157   9.630377  128.837353        None   
ber0002-0011   7.512558    79.234887   8.901398  129.482491        None   
ber0002-0012   7.210548    79.289497   8.904465  131.010602        None   
ber0002-0013   7.547493    95.120607   9.752979  136.403294        None   

                                                       geometry  
id                                                               
aus0001-0000  LINESTRING (153.26555 -24.7007, 153.26938 -24....  
aus0001-0001  LINESTRING (153.26525 -24.7019, 153.2692 -24.7...  
aus0001-0002  LINESTRING (153.26539 -24.70316, 153.26931 -24...  
aus0001-0003  LINESTRING (153.26555 -24.70408, 153.26945 -24...  
aus0001-0004  LINESTRING (153.2657 -24.70497, 153.26961 -24....  
...                                                         ...  
ber0002-0009  LINESTRING (-64.82204 32.25336, -64.82017 32.2...  
ber0002-0010  LINESTRING (-64.82143 32.25361, -64.82029 32.2...  
ber0002-0011  LINESTRING (-64.82118 32.25369, -64.82004 32.2...  
ber0002-0012  LINESTRING (-64.82049 32.25382, -64.82022 32.2...  
ber0002-0013  LINESTRING (-64.82022 32.25384, -64.81995 32.2...  

[147216 rows x 17 columns]

In [36]:
trends.columns, transects.columns, trends.columns.isin(transects.columns)

(Index(['trend', 'intercept', 'n_points', 'n_points_nonan', 'r2_score', 'mae',
        'mse', 'rmse'],
       dtype='object'),
 Index(['site_id', 'orientation', 'along_dist', 'along_dist_norm',
        'beach_slope', 'cil', 'ciu', 'trend', 'n_points', 'n_points_nonan',
        'r2_score', 'mae', 'mse', 'rmse', 'intercept', 'ERODIBILITY',
        'geometry'],
       dtype='object'),
 array([ True,  True,  True,  True,  True,  True,  True,  True]))

In [37]:
transects = transects.join(trends.loc[:, ~trends.columns.isin(transects.columns)])
transects

site_id  orientation  along_dist  along_dist_norm  beach_slope  \
id                                                                             
aus0001-0000  aus0001   104.347648    0.000000         0.000000        0.085   
aus0001-0001  aus0001    93.495734   98.408334         0.002935        0.050   
aus0001-0002  aus0001    82.069341  198.408334         0.005918        0.050   
aus0001-0003  aus0001    81.192757  298.402523         0.008900        0.055   
aus0001-0004  aus0001    81.065473  398.402523         0.011882        0.075   
...               ...          ...         ...              ...          ...   
ber0002-0009  ber0002          NaN         NaN              NaN          NaN   
ber0002-0010  ber0002          NaN         NaN              NaN          NaN   
ber0002-0011  ber0002          NaN         NaN              NaN          NaN   
ber0002-0012  ber0002          NaN         NaN              NaN          NaN   
ber0002-0013  ber0002          NaN         NaN              NaN          NaN   

                 cil     ciu     trend  n_points  n_points_nonan  r2_score  \
id                                                                           
aus0001-0000  0.0545  0.2000 -1.441081     767.0           428.0  0.168420   
aus0001-0001  0.0387  0.0640 -1.037105     767.0           569.0  0.097874   
aus0001-0002  0.0428  0.0647 -0.680019     767.0           588.0  0.053927   
aus0001-0003  0.0480  0.0659 -0.405198     767.0           598.0  0.023412   
aus0001-0004  0.0614  0.0922 -0.090025     767.0           608.0  0.001277   
...              ...     ...       ...       ...             ...       ...   
ber0002-0009     NaN     NaN  0.088578     223.0           192.0  0.006440   
ber0002-0010     NaN     NaN  0.028300     223.0           190.0  0.000497   
ber0002-0011     NaN     NaN  0.057356     223.0           187.0  0.002423   
ber0002-0012     NaN     NaN -0.010553     223.0           186.0  0.000081   
ber0002-0013     NaN     NaN  0.157248     223.0           177.0  0.014649   

                    mae          mse       rmse   intercept ERODIBILITY  \
id                                                                        
aus0001-0000  28.102591  1263.560863  35.546601  179.085729        None   
aus0001-0001  25.419324  1033.770813  32.152306  212.247788        None   
aus0001-0002  22.632907   838.007507  28.948359  205.106151        None   
aus0001-0003  20.749758   698.653187  26.432048  191.745881        None   
aus0001-0004  19.889328   655.810616  25.608800  175.092121        None   
...                 ...          ...        ...         ...         ...   
ber0002-0009   6.645872    70.165869   8.376507  128.134846        None   
ber0002-0010   7.682399    92.744157   9.630377  128.837353        None   
ber0002-0011   7.512558    79.234887   8.901398  129.482491        None   
ber0002-0012   7.210548    79.289497   8.904465  131.010602        None   
ber0002-0013   7.547493    95.120607   9.752979  136.403294        None   

                                                       geometry  
id                                                               
aus0001-0000  LINESTRING (153.26555 -24.7007, 153.26938 -24....  
aus0001-0001  LINESTRING (153.26525 -24.7019, 153.2692 -24.7...  
aus0001-0002  LINESTRING (153.26539 -24.70316, 153.26931 -24...  
aus0001-0003  LINESTRING (153.26555 -24.70408, 153.26945 -24...  
aus0001-0004  LINESTRING (153.2657 -24.70497, 153.26961 -24....  
...                                                         ...  
ber0002-0009  LINESTRING (-64.82204 32.25336, -64.82017 32.2...  
ber0002-0010  LINESTRING (-64.82143 32.25361, -64.82029 32.2...  
ber0002-0011  LINESTRING (-64.82118 32.25369, -64.82004 32.2...  
ber0002-0012  LINESTRING (-64.82049 32.25382, -64.82022 32.2...  
ber0002-0013  LINESTRING (-64.82022 32.25384, -64.81995 32.2...  

[147216 rows x 17 columns]

In [38]:
transects[transects.site_id.str.startswith("sar") & ~transects.trend.isna()]

site_id  orientation  along_dist  along_dist_norm  beach_slope  \
id                                                                             
sar0001-0000  sar0001          NaN         NaN              NaN          NaN   
sar0001-0001  sar0001          NaN         NaN              NaN          NaN   
sar0001-0002  sar0001          NaN         NaN              NaN          NaN   
sar0001-0003  sar0001          NaN         NaN              NaN          NaN   
sar0001-0004  sar0001          NaN         NaN              NaN          NaN   
...               ...          ...         ...              ...          ...   
sar2541-0000  sar2541          NaN         NaN              NaN          NaN   
sar2541-0001  sar2541          NaN         NaN              NaN          NaN   
sar2541-0002  sar2541          NaN         NaN              NaN          NaN   
sar2541-0003  sar2541          NaN         NaN              NaN          NaN   
sar2541-0004  sar2541          NaN         NaN              NaN          NaN   

              cil  ciu     trend  n_points  n_points_nonan  r2_score  \
id                                                                     
sar0001-0000  NaN  NaN -0.209107     671.0           667.0  0.003085   
sar0001-0001  NaN  NaN -0.168722     671.0           667.0  0.008700   
sar0001-0002  NaN  NaN -0.012318     671.0           669.0  0.000413   
sar0001-0003  NaN  NaN -0.043548     671.0           669.0  0.016188   
sar0001-0004  NaN  NaN  0.011825     671.0           669.0  0.001501   
...           ...  ...       ...       ...             ...       ...   
sar2541-0000  NaN  NaN -0.297976    1371.0          1371.0  0.203601   
sar2541-0001  NaN  NaN  0.011742    1371.0          1371.0  0.000671   
sar2541-0002  NaN  NaN  0.048766    1371.0          1371.0  0.008370   
sar2541-0003  NaN  NaN -0.212156    1371.0          1371.0  0.135324   
sar2541-0004  NaN  NaN -1.424914    1371.0          1371.0  0.750608   

                    mae          mse       rmse   intercept ERODIBILITY  \
id                                                                        
sar0001-0000  34.337637  2024.899412  44.998882  135.992639      Medium   
sar0001-0001  11.931958   460.730301  21.464629  197.877604      Medium   
sar0001-0002   5.801157    51.994338   7.210710  206.410952      Medium   
sar0001-0003   3.202754    16.331152   4.041182  239.500879      Medium   
sar0001-0004   2.692829    13.178444   3.630213  258.136097      Medium   
...                 ...          ...        ...         ...         ...   
sar2541-0000   5.662483    50.986351   7.140473  199.127124        High   
sar2541-0001   3.998349    30.150238   5.490923  165.933848        High   
sar2541-0002   4.829729    41.363000   6.431407  161.418975        High   
sar2541-0003   5.148677    42.220947   6.497765  182.397918        High   
sar2541-0004   7.903430    99.034737   9.951620  416.753361        High   

                                                     geometry  
id                                                             
sar0001-0000  LINESTRING (8.40852 38.86175, 8.40882 38.86535)  
sar0001-0001   LINESTRING (8.4084 38.86162, 8.41092 38.86464)  
sar0001-0002  LINESTRING (8.40893 38.86153, 8.41236 38.86393)  
sar0001-0003   LINESTRING (8.40904 38.86129, 8.41333 38.8626)  
sar0001-0004   LINESTRING (8.40904 38.86176, 8.4133 38.86037)  
...                                                       ...  
sar2541-0000  LINESTRING (8.85399 38.88006, 8.85388 38.87736)  
sar2541-0001  LINESTRING (8.85428 38.88005, 8.85417 38.87735)  
sar2541-0002  LINESTRING (8.85502 38.87993, 8.85398 38.87735)  
sar2541-0003   LINESTRING (8.85536 38.87984, 8.85418 38.8773)  
sar2541-0004   LINESTRING (8.85656 38.8791, 8.85349 38.87785)  

[19763 rows x 17 columns]

In [39]:
transects.drop_duplicates().to_file("transects_extended.geojson", driver="GeoJSON")